In [1]:
import numpy as np
from gymnasium import spaces
from panda_gym import env_id
from panda_gym.envs.core import PyBulletRobot
from panda_gym.pybullet import PyBullet
from PIL import Image

pybullet build time: Sep  3 2024 12:59:12


In [2]:
from typing import Optional

import numpy as np
from gymnasium import spaces

from panda_gym.envs.core import PyBulletRobot
from panda_gym.pybullet import PyBullet


class UR5(PyBulletRobot):
    """ UR5 robot in PyBullet.

    Args:
        sim (PyBullet): Simulation instance.
        block_gripper (bool, optional): Whether the gripper is blocked. Defaults to False.
        base_position (np.ndarray, optional): Position of the base of the robot, as (x, y, z). Defaults to (0, 0, 0).
        control_type (str, optional): "ee" to control end-effector displacement or "joints" to control joint angles.
            Defaults to "ee".
    """

    def __init__(
        self,
        sim: PyBullet,
        block_gripper: bool = False,
        base_position: Optional[np.ndarray] = None,
        control_type: str = "ee",
    ) -> None:
        base_position = base_position if base_position is not None else np.zeros(3)
        self.block_gripper = block_gripper
        self.control_type = control_type
        n_action = 3 if self.control_type == "ee" else 6  # control (x, y z) if "ee", else, control the 7 joints
        n_action += 0 if self.block_gripper else 1
        action_space = spaces.Box(-1.0, 1.0, shape=(n_action,), dtype=np.float32)
        super().__init__(
            sim,
            body_name="ur5_robot",
            file_name="ur_5_robot/urdf/ur5_with_gripper.urdf",
            base_position=base_position,
            action_space=action_space,
            joint_indices=np.array([1, 2, 3, 4, 5, 6, 10, 11]),
            joint_forces=np.array([150.0, 150.0, 150.0, 28.0, 28.0, 28.0, 170.0, 170.0]),
        )

        self.fingers_indices = np.array([10, 11])
        self.neutral_joint_values = np.deg2rad(np.array([180.0, -90.0, -90.0, -90.0, 90.0, -90.0, 0.00, 0.00]))
        self.ee_link = 12
        self.sim.set_lateral_friction(self.body_name, self.fingers_indices[0], lateral_friction=1.0)
        self.sim.set_lateral_friction(self.body_name, self.fingers_indices[1], lateral_friction=1.0)
        self.sim.set_spinning_friction(self.body_name, self.fingers_indices[0], spinning_friction=0.001)
        self.sim.set_spinning_friction(self.body_name, self.fingers_indices[1], spinning_friction=0.001)

    def set_action(self, action: np.ndarray) -> None:
        action = action.copy()  # ensure action don't change
        action = np.clip(action, self.action_space.low, self.action_space.high)
        if self.control_type == "ee":
            ee_displacement = action[:3]
            target_arm_angles = self.ee_displacement_to_target_arm_angles(ee_displacement)
        else:
            arm_joint_ctrl = action[:6]
            target_arm_angles = self.arm_joint_ctrl_to_target_arm_angles(arm_joint_ctrl)

        if self.block_gripper:
            target_fingers_width = 0
        else:
            fingers_ctrl = action[-1] * 0.2  # limit maximum change in position
            fingers_width = self.get_fingers_width()
            target_fingers_width = fingers_width + fingers_ctrl

        target_angles = np.concatenate((target_arm_angles, [target_fingers_width / 2, target_fingers_width / 2]))
        self.control_joints(target_angles=target_angles)

    def ee_displacement_to_target_arm_angles(self, ee_displacement: np.ndarray) -> np.ndarray:
        """Compute the target arm angles from the end-effector displacement.

        Args:
            ee_displacement (np.ndarray): End-effector displacement, as (dx, dy, dy).

        Returns:
            np.ndarray: Target arm angles, as the angles of the 7 arm joints.
        """
        ee_displacement = ee_displacement[:3] * 0.05  # limit maximum change in position
        # get the current position and the target position
        ee_position = self.get_ee_position()
        target_ee_position = ee_position + ee_displacement
        # Clip the height target. For some reason, it has a great impact on learning
        target_ee_position[2] = np.max((0, target_ee_position[2]))
        # compute the new joint angles
        target_arm_angles = self.inverse_kinematics(
            link=self.ee_link, position=target_ee_position, orientation=np.array([1.0, 0.0, 0.0, 0.0])
        )
        target_arm_angles = target_arm_angles[:6]  # remove fingers angles
        return target_arm_angles

    def arm_joint_ctrl_to_target_arm_angles(self, arm_joint_ctrl: np.ndarray) -> np.ndarray:
        """Compute the target arm angles from the arm joint control.

        Args:
            arm_joint_ctrl (np.ndarray): Control of the 7 joints.

        Returns:
            np.ndarray: Target arm angles, as the angles of the 7 arm joints.
        """
        arm_joint_ctrl = arm_joint_ctrl * 0.05  # limit maximum change in position
        # get the current position and the target position
        current_arm_joint_angles = np.array([self.get_joint_angle(joint=i) for i in range(1, 7)])
        target_arm_angles = current_arm_joint_angles + arm_joint_ctrl
        return target_arm_angles

    def get_obs(self) -> np.ndarray:
        # end-effector position and velocity
        ee_position = np.array(self.get_ee_position())
        ee_velocity = np.array(self.get_ee_velocity())
        # fingers opening
        if not self.block_gripper:
            fingers_width = self.get_fingers_width()
            observation = np.concatenate((ee_position, ee_velocity, [fingers_width]))
        else:
            observation = np.concatenate((ee_position, ee_velocity))
        return observation

    def reset(self) -> None:
        self.set_joint_neutral()

    def set_joint_neutral(self) -> None:
        """Set the robot to its neutral pose."""
        self.set_joint_angles(self.neutral_joint_values)

    def get_fingers_width(self) -> float:
        """Get the distance between the fingers."""
        finger1 = self.sim.get_joint_angle(self.body_name, self.fingers_indices[0])
        finger2 = self.sim.get_joint_angle(self.body_name, self.fingers_indices[1])
        return finger1 + finger2

    def get_ee_position(self) -> np.ndarray:
        """Returns the position of the end-effector as (x, y, z)"""
        return self.get_link_position(self.ee_link)

    def get_ee_velocity(self) -> np.ndarray:
        """Returns the velocity of the end-effector as (vx, vy, vz)"""
        return self.get_link_velocity(self.ee_link)

In [4]:
from panda_gym.envs.core import RobotTaskEnv
from panda_gym.envs.tasks.pick_and_place import PickAndPlace
from panda_gym.envs.tasks.stack import Stack

In [4]:
env_render_options = {
    "render_width": 480,
    "render_height": 480,
    "render_target_position": [-0.2, 0, 0],
    "render_distance": 1.2,
    "render_yaw": 90,
    "render_pitch": -30,
    "render_roll": 0,
}

In [5]:
class UR5StackEnv(RobotTaskEnv):
    """Pick and Place task wih Panda robot.

    Args:
        render_mode (str, optional): Render mode. Defaults to "rgb_array".
        reward_type (str, optional): "sparse" or "dense". Defaults to "sparse".
        control_type (str, optional): "ee" to control end-effector position or "joints" to control joint values.
            Defaults to "ee".
        renderer (str, optional): Renderer, either "Tiny" or OpenGL". Defaults to "Tiny" if render mode is "human"
            and "OpenGL" if render mode is "rgb_array". Only "OpenGL" is available for human render mode.
        render_width (int, optional): Image width. Defaults to 720.
        render_height (int, optional): Image height. Defaults to 480.
        render_target_position (np.ndarray, optional): Camera targeting this position, as (x, y, z).
            Defaults to [0., 0., 0.].
        render_distance (float, optional): Distance of the camera. Defaults to 1.4.
        render_yaw (float, optional): Yaw of the camera. Defaults to 45.
        render_pitch (float, optional): Pitch of the camera. Defaults to -30.
        render_roll (int, optional): Roll of the camera. Defaults to 0.
    """

    def __init__(
        self,
        render_mode: str = "rgb_array",
        reward_type: str = "sparse",
        control_type: str = "ee",
        renderer: str = "Tiny",
        render_width: int = 720,
        render_height: int = 480,
        render_target_position: Optional[np.ndarray] = None,
        render_distance: float = 1.4,
        render_yaw: float = 45,
        render_pitch: float = -30,
        render_roll: float = 0,
    ) -> None:
        sim = PyBullet(render_mode=render_mode, renderer=renderer)
        robot = UR5(sim, block_gripper=False, base_position=np.array([-0.6, 0.0, 0.0]), control_type=control_type)
        task = Stack(sim, reward_type=reward_type)
        super().__init__(
            robot,
            task,
            render_width=render_width,
            render_height=render_height,
            render_target_position=render_target_position,
            render_distance=render_distance,
            render_yaw=render_yaw,
            render_pitch=render_pitch,
            render_roll=render_roll,
        )


In [6]:
import gymnasium as gym
import panda_gym

env = UR5StackEnv(render_mode="rgb_array", **env_render_options, control_type='joints')

observation, info = env.reset()

images = [env.render()]
for _ in range(100):
    action = env.action_space.sample() # random action
    observation, reward, terminated, truncated, info = env.step(action)
    images.append(env.render())

    if terminated or truncated:
        observation, info = env.reset()

env.close()

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
base_linkb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
baseb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
flangeb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data

In [7]:
frames = [Image.fromarray(img) for img in images]
frames[0].save(
        f"ur5_stack_color.gif",
        save_all=True, append_images=frames[1:],
        duration=100, loop=0
    )

In [113]:
env_render_options = {
    "render_width": 480,
    "render_height": 720,
    "render_target_position": [-0.5, 0.3, 0.5],
    "render_distance": 2,
    "render_yaw": -25,
    "render_pitch": -10,
    "render_roll": 0,
}

In [119]:
import gymnasium as gym
import panda_gym
angles = np.deg2rad(np.array([90, -90, -25, -65, 90, 90, 1.5, 1.5]))
env = UR5StackEnv(render_mode="rgb_array", **env_render_options, control_type='joints')
observation, info = env.reset()
env.robot.set_joint_angles(angles)
img = env.render()
Image.fromarray(img).save(f"ur5_demonstration.png")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
base_linkb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
baseb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
flangeb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data

# Making frames and gifs

In [5]:
class UR5PickAndPlaceEnv(RobotTaskEnv):
    """Pick and Place task wih Panda robot.

    Args:
        render_mode (str, optional): Render mode. Defaults to "rgb_array".
        reward_type (str, optional): "sparse" or "dense". Defaults to "sparse".
        control_type (str, optional): "ee" to control end-effector position or "joints" to control joint values.
            Defaults to "ee".
        renderer (str, optional): Renderer, either "Tiny" or OpenGL". Defaults to "Tiny" if render mode is "human"
            and "OpenGL" if render mode is "rgb_array". Only "OpenGL" is available for human render mode.
        render_width (int, optional): Image width. Defaults to 720.
        render_height (int, optional): Image height. Defaults to 480.
        render_target_position (np.ndarray, optional): Camera targeting this position, as (x, y, z).
            Defaults to [0., 0., 0.].
        render_distance (float, optional): Distance of the camera. Defaults to 1.4.
        render_yaw (float, optional): Yaw of the camera. Defaults to 45.
        render_pitch (float, optional): Pitch of the camera. Defaults to -30.
        render_roll (int, optional): Roll of the camera. Defaults to 0.
    """

    def __init__(
        self,
        render_mode: str = "rgb_array",
        reward_type: str = "sparse",
        control_type: str = "ee",
        renderer: str = "Tiny",
        render_width: int = 720,
        render_height: int = 480,
        render_target_position: Optional[np.ndarray] = None,
        render_distance: float = 1.4,
        render_yaw: float = 45,
        render_pitch: float = -30,
        render_roll: float = 0,
    ) -> None:
        sim = PyBullet(render_mode=render_mode, renderer=renderer)
        robot = UR5(sim, block_gripper=False, base_position=np.array([-0.6, 0.0, 0.0]), control_type=control_type)
        task = PickAndPlace(sim, reward_type=reward_type)
        super().__init__(
            robot,
            task,
            render_width=render_width,
            render_height=render_height,
            render_target_position=render_target_position,
            render_distance=render_distance,
            render_yaw=render_yaw,
            render_pitch=render_pitch,
            render_roll=render_roll,
        )


In [10]:
env_render_options = {
    "render_width": 480,
    "render_height": 480,
    "render_target_position": [-0.4, 0.4, 0],
    "render_distance": 1.65,
    "render_yaw": 0,
    "render_pitch": -30,
    "render_roll": 0,
}

In [ ]:
env = UR5PickAndPlaceEnv(render_mode="rgb_array", **env_render_options, control_type='joints')

angle_step = 0.25

for j in range(1, 7):
    observation, info = env.reset()
    images = [env.render()]
    for i in range(100):
        action = np.zeros(env.action_space.shape)
        action[j - 1] = -angle_step
        observation, reward, terminated, truncated, info = env.step(action)
        images.append(env.render())

    frames = [Image.fromarray(img) for img in images]
    frames[0].save(
        f"demo/joint_{j}.gif",
        save_all=True, append_images=frames[1:],
        duration=100, loop=0
    )

In [13]:
observation, info = env.reset()
j = 7
gripper_step = 0.01
default_angles = env.robot.neutral_joint_values
default_angles[j - 2] -= np.pi / 4
env.robot.set_joint_angles(default_angles)

images = [env.render()]
for i in range(100):
    action = np.zeros(env.action_space.shape)
    action[j - 1] = gripper_step
    observation, reward, terminated, truncated, info = env.step(action)
    images.append(env.render())

frames = [Image.fromarray(img) for img in images]
frames[0].save(
    f"demo/gripper.gif",
    save_all=True, append_images=frames[1:],
    duration=100, loop=0
)

In [62]:
env_render_options = {
    "render_width": 480,
    "render_height": 480,
    "render_target_position": [-0.2, 0, 0],
    "render_distance": 1.2,
    "render_yaw": 90,
    "render_pitch": -30,
    "render_roll": 0,
}

In [66]:
import gymnasium as gym
import panda_gym

env = UR5PickAndPlaceEnv(render_mode="rgb_array", **env_render_options, control_type='joints')

observation, info = env.reset()

images = [env.render()]
for _ in range(100):
    action = env.action_space.sample() # random action
    observation, reward, terminated, truncated, info = env.step(0.5 * action)
    images.append(env.render())

    if terminated or truncated:
        observation, info = env.reset()

env.close()

frames = [Image.fromarray(img) for img in images]
frames[0].save(
    f"demo/moving.gif",
    save_all=True, append_images=frames[1:],
    duration=100, loop=0
)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
base_linkb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
baseb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
flangeb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data

In [67]:
env = UR5PickAndPlaceEnv(render_mode="rgb_array", **env_render_options, control_type='ee')
start_position = env.robot.get_ee_position()
observation, info = env.reset()
images = [env.render()]
for i in range(100):
    if i < 25:
        gripper_step = 0
        coords_action = observation['achieved_goal'] - env.robot.get_ee_position()
        coords_action[2] = 0
    elif i > 25 and i < 50:
        gripper_step = 0.1
        coords_action = observation['achieved_goal'] - env.robot.get_ee_position()
        coords_action[2] -= 0.0001
        print(env.robot.get_ee_position())
    elif i > 50 and i < 75:
        gripper_step = -0.1
        coords_action = np.zeros(3)
    else:
        gripper_step = 0
        coords_action = start_position - env.robot.get_ee_position()
    action = np.concatenate((3 * coords_action, np.array([gripper_step])))
    observation, reward, terminated, truncated, info = env.step(action)
    images.append(env.render())

frames = [Image.fromarray(img) for img in images]
frames[0].save(
    f"demo/moving.gif",
    save_all=True, append_images=frames[1:],
    duration=100, loop=0
)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
base_linkb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
baseb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
flangeb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data

In [68]:
env.sim.get_link_position("ur5_robot", 12)

array([-0.1080284 , -0.10167229,  0.31185585])

In [69]:
env.sim.get_link_position("ur5_robot", 11)

array([-0.10802813, -0.07003352,  0.3584559 ])

In [70]:
env.sim.get_link_position("ur5_robot", 10)


array([-0.10802851, -0.13331202,  0.35845579])

In [83]:
observation['achieved_goal']

array([ 0.11317387, -0.04439889,  0.01998941], dtype=float32)

In [72]:
observation['desired_goal']

array([0.13937998, 0.02813193, 0.12348087], dtype=float32)

In [26]:
np.mean([env.sim.get_link_position("ur5_robot", 10), env.sim.get_link_position("ur5_robot", 11)], axis=0)

array([-0.11703223, -0.10254358,  0.3518938 ])

In [160]:
env.sim.get_link_position("ur5_robot", 9)


array([-0.19024541, -0.08289785,  0.76379192])

In [148]:
env.sim.get_joint_position("ur5_robot", 9)


AttributeError: 'PyBullet' object has no attribute 'get_joint_position'

In [84]:
env = UR5PickAndPlaceEnv(render_mode="rgb_array", **env_render_options, control_type='ee')
start_position = env.robot.get_ee_position()
observation, info = env.reset()
images = [env.render()]
for i in range(100):
    gripper_step = 0.05
    coords_action = np.array([0.2, 0.1, 0.01]) - env.robot.get_ee_position()
    action = np.concatenate((3 * coords_action, np.array([gripper_step])))
    observation, reward, terminated, truncated, info = env.step(0.1 * action)
    images.append(env.render())

frames = [Image.fromarray(img) for img in images]
frames[0].save(
    f"demo/moving.gif",
    save_all=True, append_images=frames[1:],
    duration=100, loop=0
)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
base_linkb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
baseb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
flangeb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data

In [87]:
# extract_keyframes.py
from pathlib import Path
from PIL import Image, ImageSequence

SRC_DIR       = Path("demo")          # where your 8 GIFs live
DST_DIR       = Path("demo")          # put extracted folders here
KEY_IDXS      = [0, 15, 30, 45]       # which frames to keep

for gif_path in SRC_DIR.glob("*.gif"):
    with Image.open(gif_path) as im:
        out_dir = DST_DIR / gif_path.stem           # e.g. gifs/ur5_joint_0
        out_dir.mkdir(parents=True, exist_ok=True)

        for k in KEY_IDXS:
            try:
                im.seek(k)                          # jump to frame k
            except EOFError:                        # GIF shorter than k
                break
            frame = im.convert("RGB")               # drop palette
            frame.save(out_dir / f"frame_{k:03d}.png")

        print(f"✓ {gif_path.name}: saved {len(list(out_dir.glob('*.png')))} frames")


✓ joint_6.gif: saved 4 frames
✓ joint_4.gif: saved 4 frames
✓ joint_5.gif: saved 4 frames
✓ joint_1.gif: saved 4 frames
✓ joint_2.gif: saved 4 frames
✓ joint_3.gif: saved 4 frames
✓ gripper.gif: saved 4 frames
✓ moving.gif: saved 4 frames
